Here, I demonstrate how to use predeval with a model producing categorical outputs. This example uses the [iris dataset from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). I start by loading this dataset and using the first 100 samples to train a logistic regression model.

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import numpy as np
from numpy.random import seed

# load data
iris = datasets.load_iris()
all_x = iris.data[:, :2]
all_y = iris.target

# shuffle data
indices = np.arange(all_x.shape[0])
seed(1234)
np.random.shuffle(indices)

# create training set
X = all_x[indices[:100], :]
Y = all_y[indices[:100]]

# train model
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
logreg.fit(X, Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)

Now that we have a trained model, we can use the model's output to create a CategoricalEvaluator object. We start by importing the CategoricalEvaluator class and instantiating an object, <code>cat_ce</code>, with the model's output. Predeval uses this data to form expectations about how future data will look.

In [3]:
from predeval import CategoricalEvaluator

# give model output from training data to CategoricalEvaluator object
model_output = logreg.predict(X)
cat_ce = CategoricalEvaluator(model_output)

In a real setting, we would give the model completely new data and pass its new predictions into `cat_ce`. Instead, we will pass the remaining data to the model. We will then ask predeval to compare these new outputs to the old outputs.

In [4]:
# give validation data to CateogoricalEvaluator object
new_model_output = logreg.predict(all_x[indices[100:], :])
cat_ce.check_data(new_model_output)

Passed exist check; observed=[0 1 2] (Expected [0, 1, 2])
Passed chi2 check; test statistic=0.6065, p=0.7384


[('exist', True), ('chi2', True)]

Not surprisingly, predeval does not detect a change in the model's outputs.

We can also give the model's output probabilities to a ContinuousEvaluator object.

In [5]:
from predeval import ContinuousEvaluator

# give probabilities from training data to ContinuousEvaluator object
new_model_output = logreg.predict_proba(X)
con_ce = ContinuousEvaluator(new_model_output[:, 1])

Again, we can pass the remaining samples to the ContinuousEvaluator.

In [6]:
# give validation data to ContinuousEvaluator object
new_model_output = logreg.predict_proba(all_x[100:, :])
con_ce.check_data(new_model_output[:, 1])

Passed min check; min observed=0.0121
Passed max check; max observed=0.9315
Passed mean check; mean observed=0.3838 (Expected 0.3500 +- 0.6873)
Passed std check; std observed=0.2692 (Expected 0.3437 +- 0.1718)
Passed ks check; test statistic=0.3400, p=0.0006


[('min', True), ('max', True), ('mean', True), ('std', True), ('ks', True)]

Again, predeval does not detect any changes in the model's output. 

Let's say one of your features becomes corrupted and is populated entirely by 1s.

In [7]:
all_x[:, 0] = 1
new_bad_model_output = logreg.predict(all_x[indices[100:], :])
cat_ce.check_data(new_bad_model_output)

Failed exist check; observed=[0] (Expected [0, 1, 2])
Failed chi2 check; test statistic=1000.0000, p=0.0000


[('exist', False), ('chi2', False)]

In this case, we predeval does detect a change in the model's output.

In [8]:
new_model_output = logreg.predict_proba(all_x[100:, :])
con_ce.check_data(new_model_output[:, 1])

Failed min check; min observed=0.0000
Passed max check; max observed=0.0000
Passed mean check; mean observed=0.0000 (Expected 0.3500 +- 0.6873)
Failed std check; std observed=0.0000 (Expected 0.3437 +- 0.1718)
Failed ks check; test statistic=1.0000, p=0.0000


[('min', False), ('max', True), ('mean', True), ('std', False), ('ks', False)]

Again, predeval detects a change in the model's outputs.